In [17]:
import pandas as pd

In [18]:
school_data = pd.read_csv("Resources/schools_complete.csv")
student_data = pd.read_csv("Resources/students_complete.csv")

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  
0                 66          79          0  District  2917  1910635  
1                 94          61          0  District  2917  1910635  
2                 90          60          0  District  2917  1910635  
3                 67          58          0  District  2917  1910635  
4                 97          84          0  District  2917  1910635  
...              ...         ...        ...       ...   ...      ...  
39165             99          90         14   Charter  1635  1043130  
39166             95          70         14   Charter  1635  1043130  
39167             73          84         14   Charter  1635  1043130  
39168             99          90         14   Charter  1635  1043130  
39169             95          75         14   Charter  1635  1043130  

[39170 rows x 11 columns]

In [19]:
#District Summary

unique_school_names = school_data['school_name'].unique()

school_count = len(unique_school_names)

dist_student_count = school_data['size'].sum()

total_student_rec = student_data['student_name'].count()

total_budget = school_data['budget'].sum()

num_passing_reading = student_data.loc[student_data['reading_score'] >= 70]['reading_score'].count()
perc_pass_reading = num_passing_reading/total_student_rec

num_passing_math = student_data.loc[student_data['math_score'] >= 70]['math_score'].count()
perc_pass_math = num_passing_math/total_student_rec

average_math_score = student_data['math_score'].mean()

average_reading_score = student_data['reading_score'].mean()

overall_passing_score = (average_math_score + average_reading_score)/2

district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [dist_student_count],
    "Total Budget": [total_budget],
    "Avg Reading Score": [average_reading_score],
    "Avg Math Score": [average_math_score],
    "% Passing Reading":[perc_pass_reading],
    "% Passing Math": [perc_pass_math],
    "Overall Passing Rate": [overall_passing_score]

})

dist_sum = district_summary[["Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Avg Math Score", 
                             "Avg Reading Score", 
                             '% Passing Math', 
                             '% Passing Reading', 
                             'Overall Passing Rate']]

dist_sum.style.format({"Total Budget": "${:,.2f}", 
                       "Avg Math Score": "{:.1f}", 
                       "Avg Reading Score": "{:.1f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})

In [20]:
#School Summary

by_school = school_data_complete.set_index('school_name').groupby(['school_name'])

sch_types = school_data.set_index('school_name')['type']

stu_per_sch = by_school['Student ID'].count()

sch_budget = school_data.set_index('school_name')['budget']

stu_budget = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

avg_math = by_school['math_score'].mean()
avg_read = by_school['reading_score'].mean()

pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_sch 

school_summary = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Per Student Budget": stu_budget,
    "Total School Budget": sch_budget,
    "Avg Math Score": avg_math,
    "Avg Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
})

school_summary = school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Avg Math Score', 
                          'Avg Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

school_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Avg Math Score': "{:.1f}", 
                          'Avg Reading Score': "{:.1f}", 
                          "% Passing Math": "{:.1%}", 
                          "% Passing Reading": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

In [21]:
#Top Performing Schools (By Passing Rate)

top_5 = school_summary.sort_values("Overall Passing Rate", ascending = False)
top_5.head().style.format({'Total Students': '{:,}',
                           "Total School Budget": "${:,}", 
                           "Per Student Budget": "${:.0f}", 
                           "% Passing Math": "{:.1%}", 
                           "% Passing Reading": "{:.1%}", 
                           "Overall Passing Rate": "{:.1%}"})

In [22]:
#Bottom Performing Schools (By Passing Rate)

bottom_5 = top_5.tail()
bottom_5 = bottom_5.sort_values('Overall Passing Rate')
bottom_5.style.format({'Total Students': '{:,}', 
                       "Total School Budget": "${:,}", 
                       "Per Student Budget": "${:.0f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})

In [23]:
#Math Scores by Grade

ninth_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School"

math_scores.style.format({'9th': '{:.1f}', 
                          "10th": '{:.1f}', 
                          "11th": "{:.1f}", 
                          "12th": "{:.1f}"})

In [24]:
#Reading Scores by Grade

ninth_reading = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_reading = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_reading = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_reading = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School"

reading_scores.style.format({'9th': '{:.1f}', 
                             "10th": '{:.1f}', 
                             "11th": "{:.1f}", 
                             "12th": "{:.1f}"})

In [25]:
#Scores by School Spending

bins = [0, 584.999, 614.999, 644.999, 999999]
group_name = ['< $585', "$585 - 614", "$615 - 644", "$645 - 675"]
school_data_complete['spending_bins'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], bins, labels = group_name)

by_spending = school_data_complete.groupby('spending_bins')

avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('spending_bins')['Student ID'].count()/by_spending['Student ID'].count()
         
scores_by_spend = pd.DataFrame({
    "Avg Math Score": avg_math,
    "Avg Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})

scores_by_spend = scores_by_spend[[
    "Avg Math Score",
    "Avg Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_spend.index.name = "Per Student Budget"
scores_by_spend = scores_by_spend.reindex(group_name)

scores_by_spend.style.format({'Avg Math Score': '{:.1f}', 
                              'Avg Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

In [26]:
#Scores by School Size

bins = [0, 999, 1999, 99999999999]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (>2000)"]
school_data_complete['size_bins'] = pd.cut(school_data_complete['size'], bins, labels = group_name)

by_size = school_data_complete.groupby('size_bins')

avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('size_bins')['Student ID'].count()/by_size['Student ID'].count()
          
scores_by_size = pd.DataFrame({
    "Avg Math Score": avg_math,
    "Avg Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall
            
})

scores_by_size = scores_by_size[[
    "Avg Math Score",
    "Avg Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]

scores_by_size.index.name = "School Size"
scores_by_size = scores_by_size.reindex(group_name)

scores_by_size.style.format({'Avg Math Score': '{:.1f}', 
                              'Avg Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

In [28]:
#Scores by School Type

by_type = school_data_complete.groupby("type")

avg_math = by_type['math_score'].mean()
avg_read = by_type['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/by_type['Student ID'].count()
         
scores_by_type = pd.DataFrame({
    "Avg Math Score": avg_math,
    "Avg Reading Score": avg_read,
    '% Passing Math': pass_math,
    '% Passing Reading': pass_read,
    "Overall Passing Rate": overall})

scores_by_type = scores_by_type[[
    "Avg Math Score",
    "Avg Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "Overall Passing Rate"
]]
scores_by_type.index.name = "School Type"

scores_by_type.style.format({'Avg Math Score': '{:.1f}', 
                              'Avg Reading Score': '{:.1f}', 
                              '% Passing Math': '{:.1%}', 
                              '% Passing Reading':'{:.1%}', 
                              'Overall Passing Rate': '{:.1%}'})

In [ ]:
#I notice that students in Charter schools are more likely to have passing scores
#Schools with less students enrolled at the school are more likely to succeed
#If you have a high population, especially over 2000, you are more likely to fail math and reading testing.
    #I think this is the case because funding is stretched thinner with more students and most likely the school is not 
    #staffed with extra teaching staff to provide educational support for these tests. I thought this, then reviewed 
    #"Scores by School Spending" chart and I was shocked when quite the opposite seems to be occuring.  It looks like 
    #the more money a school spends on the student, the less likely are to succeed.  So maybem, schools with smaller 
    #enrollment may have more resources available to them, such as staff or literature that could help them rather 
    #than having more money spent "per student".